In [1]:
import networkx as nx

In [2]:
with open("input.txt") as file:
    lines = file.read().splitlines()

In [3]:
G = nx.Graph()

G.add_node("/", **{"type" : "directory", "size" : None})
current_directory = ["/"]

for line in lines:
    current_directory_path = " > ".join(current_directory)

    # Ignore list
    if line == "$ ls":
        continue
    
    # Change directory commands
    elif line[:4] == "$ cd":
        # Reset the directory
        if line == "$ cd /":
            current_directory = ["/"]
        # Move backwards
        elif line == "$ cd ..":
            current_directory.pop()
        # Move forward
        else:
            current_directory.append(line[5:])

    else:
        # Directory node
        if line[:3] == "dir":
            node_name = current_directory_path +" > "+ line[4:]
            node_attributes = {"type" : "directory", "size" : None}
            edge_weight = None
        # File node
        else:
            node_name = current_directory_path +" > "+ line.split(" ")[1]
            node_attributes = {"type" : "file", "size" : (size:=int(line.split(" ")[0]))}
            edge_weight = size

        G.add_node(node_name, **node_attributes)
        G.add_edge(current_directory_path, node_name, weight=edge_weight)

In [4]:
nodes_with_no_size = [n for n,d in G.nodes(data=True) if d["size"] is None]

while len(nodes_with_no_size)>0:
    for node in nodes_with_no_size:
        neighbors_weight = [G.nodes[neighbor]["size"] for neighbor in nx.all_neighbors(G, node) if len(neighbor) > len(node)]
        if None not in neighbors_weight:
            G.nodes[node]["size"] = sum(neighbors_weight)
            nodes_with_no_size.remove(node)

In [5]:
part_1 = [d["size"] for n,d in G.nodes(data=True) if d["size"] <= 100000 and d["type"] == "directory"]
sum(part_1)

1297159

In [6]:
total_disk_space = 70000000
minimum_unused = 30000000
current_unused = total_disk_space - sum([d["size"] for n,d in G.nodes(data=True) if d["type"] == "file"])

In [7]:
directories = [(d["size"], n) for n,d in G.nodes(data=True) if d["type"] == "directory"]

In [8]:
for dir in sorted(directories):
    if current_unused+dir[0] >= minimum_unused:
        print(dir[0])
        break

3866390
